## Model inference

In this notebook, we will load a previously trained model, explore the learned topics, and predict topics for a paper on arXiv.

In [1]:
# imports
import sys
sys.path.insert(0, "../")
from utils import scrape_arxiv_abstract
from model import TopicModel
from dataset import ArXivDataset
from gensim.models import LdaModel
from pprint import pprint
import PyPDF2
import plotly.express as px
import pandas as pd

### Build topic model

To build a `TopicModel` object, we need to pass in as arguments the dataset used to create the model (to process new instances) and the model itself (to predict topics for the new instances).

In [4]:
# create topic model
model_path = "../models/lda_n71_p10_r929_c38.2"
dataset_path = "../object/dataset.obj"
model = TopicModel(model_path, dataset_path)

### Investigate topics

Next, let us explore the different topics learned by the model so that we can assign understandable topic names to each cluster.

In [5]:
# print topics
pprint(model.topics)

[(0,
  [('arcelormittal', 0.033330865),
   ('steel', 0.03019459),
   ('group', 0.030015005),
   ('part', 0.014616405),
   ('center', 0.013107839),
   ('production', 0.012977052),
   ('manufacturing', 0.0122522125),
   ('site', 0.012006503),
   ('industry', 0.010450107),
   ('predictive_maintenance', 0.010032005)]),
 (1,
  [('student', 0.02737938),
   ('problem', 0.014803234),
   ('work', 0.013050551),
   ('health', 0.01201993),
   ('campus', 0.011776042),
   ('team', 0.0112025),
   ('day', 0.010901919),
   ('business', 0.008437887),
   ('workshop', 0.006975648),
   ('lecture', 0.006560639)]),
 (2,
  [('system', 0.0007954321),
   ('student', 0.0006826058),
   ('problem', 0.00050364),
   ('project', 0.00042514125),
   ('industrial', 0.00040677504),
   ('campus', 0.00037737263),
   ('optimization', 0.0003535675),
   ('lecture', 0.0003527776),
   ('description', 0.00033888777),
   ('group', 0.0003321681)]),
 (3,
  [('campus', 0.036214843),
   ('student', 0.033653896),
   ('language', 0.031

In [6]:
topicName = ''
TopicsNames =[]
for i in model.topics:
    for name,prob in i[1][:5]:
        topicName = topicName + name + ' ' 
    TopicsNames.append(topicName)
    topicName = ''

In [7]:
model.set_topic_names(TopicsNames)
pprint(model.topics)

[('arcelormittal steel group part center ',
  '0.033*"arcelormittal" + 0.030*"steel" + 0.030*"group" + 0.015*"part" + '
  '0.013*"center" + 0.013*"production" + 0.012*"manufacturing" + 0.012*"site" '
  '+ 0.010*"industry" + 0.010*"predictive_maintenance"'),
 ('student problem work health campus ',
  '0.027*"student" + 0.015*"problem" + 0.013*"work" + 0.012*"health" + '
  '0.012*"campus" + 0.011*"team" + 0.011*"day" + 0.008*"business" + '
  '0.007*"workshop" + 0.007*"lecture"'),
 ('system student problem project industrial ',
  '0.001*"system" + 0.001*"student" + 0.001*"problem" + 0.000*"project" + '
  '0.000*"industrial" + 0.000*"campus" + 0.000*"optimization" + '
  '0.000*"lecture" + 0.000*"description" + 0.000*"group"'),
 ('campus student language skill group ',
  '0.036*"campus" + 0.034*"student" + 0.031*"language" + 0.031*"skill" + '
  '0.019*"group" + 0.018*"level" + 0.017*"end" + 0.015*"test" + '
  '0.013*"culture" + 0.013*"semester"'),
 ('digital design electronic system student

We can see that there are some clusters that seem to refer to specific topics in machine learning. One of them is topic 7, which seems to direcly relate to sequential and time-series data. Another example is topic 10, which seems to be related to reinforcement learning.

To make it easier to refer to these topic clusters, we will assign (tentative) names to each of them. Note that these names are subject to interpretation and are only assigned to help "summarize" each cluster.

### Predict topics for a paper

Let us see how our model predicts a paper taken directly from arXiv. Using the `scrape_arxiv_abstract()` function, we can extract the title and the abstract of any paper on arXiv given its URL. Once scraped, this title and abstract can be passed into our topic model's `predict()` method.

To illustrate, let us scrape the title and abstract from the seminal paper ["Attention Is All You Need" (2017)](https://arxiv.org/abs/1706.03762) and see what topics the model detects.

In [8]:
text = "The physical systems generally rely on the fundamental concept of a feedback loop, allowing them to be controlled and giving them a behavior that is as insensitive as possible to environmental disturbances. The general objective of this course is to provide students with the concepts and skills enabling them to understand the structure and interactions within existing dynamic systems or along with their design. They will also be able to process information, design a control law to meet specifications, and analyze its performance and robustness. To achieve this, the students must first be able to define a model (or a set of models), highlighting the variables influencing the state of this system (inputs), the measures allowing access to this state, and variables to which the specifications relate (outputs), as well as the relationships between these variables. Then, in a second step, and from the analysis of the inputs that can be controlled (commands) or those that are undergone (disturbances), students will have to design a control law in order to ensure the expected performances. The last step in this course will concern the analysis of the robustness of the determined control law."

In [9]:
# get predictions
predictions = model.predict(text)

In [10]:
Top5Topics, Top5Probabilities = model.predictToChart(text)

In [11]:
fig = px.line_polar(r=Top5Probabilities, theta=Top5Topics, line_close=True)
fig.show()

/home/vitoriano/.local/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trace_data = trace_data.append(trace_data.iloc[0])


In [12]:
text2 = "The goal of this training is to let you discover the different stages of an aircraft design process in both a theoretical and a practical perspective. You will be introduced to the typical methods used in an aircraft design office, and apply this knowledge by doing the preliminary design of your own aircraft. After completing this training course, you will have acquired knowledge and skills that will enable you to work out the main aircraft characteristics and layout in a very short time frame."


In [13]:
Top5Topics, Top5Probabilities = model.predictToChart(text2)

In [14]:
fig = px.line_polar(r=Top5Probabilities, theta=Top5Topics, line_close=True)
fig.show()


/home/vitoriano/.local/lib/python3.10/site-packages/plotly/express/_core.py:271: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

